In [1]:
# Dont forget to:
# - create a directory called "cnn_models" in the directory from where this code is executed
# - put the preprocessed training data in a directory structure of "/data/pre_processed/"
# - put the preprocessed test data in a directory structure of "/data/test_pre_processed/"
from transformers import BertTokenizer, BertModel

import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

from dataset import Dataset  # import custom dataloaer

import sys
import numpy as np
import pickle

from CNNText import CNNText

import matplotlib.pyplot as plt
import time

import ipdb

ModuleNotFoundError: No module named 'ipdb'

In [ ]:
processed_train_path = "../data/pre_processed/"
train_dataset = Dataset(processed_train_path)

In [ ]:
processed_train_path = "../data/test_pre_processed/"
test_dataset = Dataset(processed_train_path)

# CNN training

In [ ]:
# initialize model
device = "cuda" if torch.cuda.is_available() else "cpu"

n_epochs = 100
lr = 0.001

kernel_num = 3
kernel_sizes = [2, 3, 4]
dropout = 0.5

cnn_classifier = CNNText(
    embed_dim=768,
    embed_num=512,
    class_num=6,
    kernel_num=kernel_num,
    kernel_sizes=kernel_sizes,
    dropout=dropout
).to(device)

optimizer = torch.optim.Adam(cnn_classifier.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
print(cnn_classifier)

In [ ]:
def pltsin(ax, x_train, x_test, y_train, y_test, ydatacolors=['b','r']):
    y = np.random.random(size=(100,1))
    ax.plot(x_train, y_train, ydatacolors[0])
    ax.plot(x_test, y_test, ydatacolors[1])
    fig.canvas.draw()

In [ ]:
def get_accuracy(output, targets):
    _, indices = output.max(1)
    accuracy = sum(indices == targets)
    return accuracy

In [ ]:
%matplotlib notebook
fig,ax = plt.subplots(1,1)
ax.set_xlabel('iteration #')
ax.set_ylabel('loss')

# to keep track of train and test performance for the plot
train_plot_x, train_plot_y, test_plot_x, test_plot_y  = [], [], [], []

iteration = 0

for epoch in range(n_epochs):
    train_loss = 0
    print(f"Current epoch {epoch} out of {n_epochs} ")
    for file_index in range(len(train_dataset)):
        iteration += 1

        # retrieve batch input and targets
        batch, labels = train_dataset[file_index]
        _, targets = labels.max(1)
            
        optimizer.zero_grad()
        cnn_classifier.train()

        output = cnn_classifier(batch.float().to(device))    
        loss = loss_fn(output, targets.to(device))
        
        train_plot_x.append(iteration)
        train_plot_y.append(loss.detach().cpu())

        loss.backward()
        optimizer.step()
        
        # evaluate performance after every 5th epoch
        if not file_index % 5:
            cnn_classifier.eval()
            test_loss = 0
            accuracy = 0
            for test_index in range(len(test_dataset)):
                test_batch, test_labels = test_dataset[test_index]
                _, test_indices = test_labels.max(1)
                output = cnn_classifier(test_batch.float().to(device))

                loss = loss_fn(output, test_indices.to(device))
                test_loss += loss.detach().cpu()
                
                accuracy += get_accuracy(output.detach().cpu(), test_indices.cpu()).item()
            
            test_plot_x.append(iteration)
            test_plot_y.append(test_loss/(len(test_dataset)))
            cnn_classifier.train()


        print(f'Epoch: [{epoch}/{n_epochs}] batch: [{file_index}/{len(train_dataset)}] Train loss: {loss} Validation loss: {test_loss/(len(test_dataset))} Validation accuracy {accuracy/(len(test_dataset))}')
        pltsin(ax, train_plot_x, test_plot_x, train_plot_y, test_plot_y, ['b', 'r'])
    torch.save(cnn_classifier.state_dict(), f"cnn_models/model_epoch_{epoch}.p")